In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')



# terus ini nama file yang kita coba buat latihan
df=pd.read_csv('/content/gdrive/My Drive/SNI/GKU - 15-10-2023.csv')

df = df.loc[:,['created_at','entry_id','field1','field2','field3','field4','field5','field6','field7']]

# HILANGIN TULISAN T DAN Z YANG ADA
df['created_at']=df['created_at'].str.replace("T"," ")
df['created_at']=df['created_at'].str.replace("Z"," ")

df.rename(columns={'field1': 'Temperatur', 'field2': 'Kelembapan Relatif','field3': 'PM2.5', 'field4' : 'CO2', 'field5' : 'Tekanan Udara','field6' : 'O3','field7' : 'CH4'}, inplace=True)
df = df.dropna()
#change to local time +7
df['created_at'] = pd.to_datetime(df['created_at']).dt.tz_convert('Asia/Jakarta')


# hilangkan nilai co2 dan pm2.5 yang bernilai <= 0
df=df[(df['CO2']>0)]
df=df[(df['PM2.5']>0)]

#
df

Mounted at /content/gdrive


,created_at,entry_id,Temperatur,Kelembapan Relatif,PM2.5,CO2,Tekanan Udara,O3,CH4
62395,2023-06-03 09:45:55+07:00,62396,13.0,2.00,796,18.00000,25.90000,76.0,93397.99219
62397,2023-06-03 09:55:07+07:00,62398,13.0,1.92,796,47.00000,26.20000,73.0,93388.64063
62398,2023-06-03 09:56:38+07:00,62399,13.0,1.92,796,47.00000,26.20000,73.0,93388.64063
62400,2023-06-03 10:12:00+07:00,62401,26.3,74.60,21,1102.15002,80246.28125,13.0,1.88000
62401,2023-06-03 10:13:30+07:00,62402,26.3,74.60,21,1102.15002,80246.28125,13.0,1.88000
...,...,...,...,...,...,...,...,...,...
187550,2023-10-15 19:32:23+07:00,187551,25.8,60.50,38,735.58002,93250.18750,0.0,1.76000
187551,2023-10-15 19:33:53+07:00,187552,25.8,60.50,38,735.58002,93250.18750,0.0,1.76000
187552,2023-10-15 19:35:23+07:00,187553,25.8,60.60,33,842.50000,93254.29688,0.0,1.76000
187553,2023-10-15 19:36:54+07:00,187554,25.8,60.60,33,842.50000,93254.29688,0.0,1.76000


In [ ]:
# Ambil Kelembapan Relatif & PM 2.5 (KITA TAMBAHKAN INDEX BULAN, HARI DAN JAM) UNTUK DISORTING NANTI
df1 = df.loc[:,['created_at','entry_id','Temperatur','Kelembapan Relatif','PM2.5','CO2']]
df1['Bulan'] = pd.DatetimeIndex(df['created_at']).month
df1.set_index = df1['Bulan'].inplace=True
df1['Hari'] = pd.DatetimeIndex(df['created_at']).day
df1.set_index = df1['Hari'].inplace=True
df1['Jam'] = pd.DatetimeIndex(df['created_at']).hour
df1.set_index = df1['Jam'].inplace=True

df1

,created_at,entry_id,Temperatur,Kelembapan Relatif,PM2.5,CO2,Bulan,Hari,Jam
62395,2023-06-03 09:45:55+07:00,62396,13.0,2.00,796,18.00000,6,3,9
62397,2023-06-03 09:55:07+07:00,62398,13.0,1.92,796,47.00000,6,3,9
62398,2023-06-03 09:56:38+07:00,62399,13.0,1.92,796,47.00000,6,3,9
62400,2023-06-03 10:12:00+07:00,62401,26.3,74.60,21,1102.15002,6,3,10
62401,2023-06-03 10:13:30+07:00,62402,26.3,74.60,21,1102.15002,6,3,10
...,...,...,...,...,...,...,...,...,...
187550,2023-10-15 19:32:23+07:00,187551,25.8,60.50,38,735.58002,10,15,19
187551,2023-10-15 19:33:53+07:00,187552,25.8,60.50,38,735.58002,10,15,19
187552,2023-10-15 19:35:23+07:00,187553,25.8,60.60,33,842.50000,10,15,19
187553,2023-10-15 19:36:54+07:00,187554,25.8,60.60,33,842.50000,10,15,19


In [ ]:
#Validasi CO2


# terus kita filter data nya per 2 menit(sesuai ama waktu register data thingspeak)
dfco2 = df1.groupby(pd.Grouper(key='created_at',freq='2min')).mean().reset_index()

#Hapus nilai CO2 yang di luar batas range
dfco2 = dfco2[(dfco2['CO2']>400) &  (dfco2['CO2'] <5000)]

# kita ambil nilai index nya sesuai yang ada di petik '',
dfco2 = dfco2.loc[:,['created_at','Temperatur','Kelembapan Relatif','CO2','Bulan','Hari','Jam']]

# dropna() fungsinya buat drop data yang kosong
dfco2 = dfco2.dropna()

# astype int ngubah angka ke int
dfco2['CO2']=dfco2['CO2'].round().astype(int)
dfco2['Bulan']=dfco2['Bulan'].round().astype(int)
dfco2['Hari']=dfco2['Hari'].round().astype(int)
dfco2['Jam']=dfco2['Jam'].round().astype(int)

#round(2) gunanya buat ngatur 2 angka dibelakang koma, bisa atur angka apa aja
dfco2

,created_at,Temperatur,Kelembapan Relatif,CO2,Bulan,Hari,Jam
14,2023-06-03 10:12:00+07:00,26.3,74.60,1102,6,3,10
15,2023-06-03 10:14:00+07:00,27.1,72.80,858,6,3,10
21,2023-06-03 10:26:00+07:00,32.8,54.55,926,6,3,10
22,2023-06-03 10:28:00+07:00,33.2,53.00,781,6,3,10
33,2023-06-03 10:50:00+07:00,32.3,53.30,629,6,3,10
...,...,...,...,...,...,...,...
96773,2023-10-15 19:30:00+07:00,25.9,60.20,858,10,15,19
96774,2023-10-15 19:32:00+07:00,25.8,60.50,736,10,15,19
96775,2023-10-15 19:34:00+07:00,25.8,60.60,842,10,15,19
96776,2023-10-15 19:36:00+07:00,25.8,60.60,842,10,15,19


In [ ]:
#Outlier windowing per jam CO2
dfoutlier = pd.DataFrame(
    {
        "created_at": ["1990-10-09 00:00:00"],
        "Temperatur": ["0"],
        "Kelembapan Relatif": ["0"],
        "CO2": ["0"],
    },
    index=[0, 1, 2, 3],
)

x=1
while x<=10 :
  y=1
  while y<=31 :
    z=0
    while z<=23 :
      dfvalidasi = dfco2[(dfco2['Bulan']==x) & (dfco2['Hari']==y) & (dfco2['Jam']==z)]
      dfvalidasi = dfvalidasi.loc[:,['created_at','Temperatur','Kelembapan Relatif','CO2']]
      data = len(dfvalidasi.axes[0])
      jumlah = str(data)
      if (dfvalidasi.empty==True):
        pass
      else :
        Q3 = dfvalidasi['CO2'].describe()['75%']
        Q1 = dfvalidasi['CO2'].describe()['25%']
        IQR = Q3 - Q1
        UF = Q3 + (1.5*IQR)
        LF = Q1 - (1.5*IQR)
        dfvalidasi = dfvalidasi[(dfvalidasi['CO2'] < UF) & (dfvalidasi['CO2'] > LF)]

        dfoutlier = pd.concat([dfoutlier, dfvalidasi], axis=0, ignore_index=True)
      z+=1
    y+=1
  x+=1

dfoutlier['CO2'] = dfoutlier['CO2'].astype(int)
dfoutlier = dfoutlier[(dfoutlier['CO2']>400) &  (dfoutlier['CO2'] <5000)]

dfoutlier

,created_at,Temperatur,Kelembapan Relatif,CO2
4,2023-03-04 13:10:00+07:00,25.2,78.1,994
5,2023-03-04 13:12:00+07:00,25.2,78.1,994
6,2023-03-04 13:14:00+07:00,25.4,76.6,947
7,2023-03-04 13:16:00+07:00,25.4,76.6,947
8,2023-03-04 13:18:00+07:00,25.6,76.9,703
...,...,...,...,...
125173,2023-10-12 04:34:00+07:00,25.1,63.4,1116
125174,2023-10-12 04:36:00+07:00,25.1,63.4,1116
125175,2023-10-12 04:38:00+07:00,25.1,63.5,781
125176,2023-10-12 04:40:00+07:00,25.1,63.5,781


In [ ]:
dfcount = dfoutlier
dfstd = dfoutlier


dfcount = dfcount.groupby(pd.Grouper(key='created_at',freq='1H')).count()[lambda x: x > 22].dropna().round().astype(int)
# ambil 1 parameter hanya untuk menghitung ketersediaan data
dfcount = dfcount['CO2'].reset_index()
dfcount = dfcount.rename(columns={'CO2':'(n)Data'})

# hitung std CO2
dfstd = dfstd.groupby(pd.Grouper(key='created_at',freq='1H')).std()
dfstd = dfstd['CO2'].reset_index().round(2)
dfstd = dfstd.rename(columns={'CO2':'σ (ppm)'})

dfdata = pd.merge(dfcount, dfstd, on='created_at', how='inner')



# ambil 1 parameter

In [ ]:
# Gabungkan data per jam hanya dengan data yang tersedia 75%
df1hco2 = dfoutlier
df1hco2 = df1hco2.groupby(pd.Grouper(key='created_at',freq='1H')).mean().round(2)

df1hco2 = pd.merge(dfdata, df1hco2, on='created_at', how='inner')

df1hco2['CV (%)'] = ((df1hco2['σ (ppm)'] / df1hco2['CO2']) * 100).round(2)

df1hco2 = df1hco2.loc[:,['created_at','(n)Data','σ (ppm)','CV (%)','Temperatur','Kelembapan Relatif','CO2','Bulan','Hari']]


df1hco2

,created_at,(n)Data,σ (ppm),CV (%),Temperatur,Kelembapan Relatif,CO2,Bulan,Hari
0,2023-06-03 11:00:00+07:00,30,70.89,11.20,34.78,45.43,633.20,6.0,3.0
1,2023-06-03 12:00:00+07:00,30,59.03,8.96,35.34,42.35,658.90,6.0,3.0
2,2023-06-03 13:00:00+07:00,30,55.90,8.32,35.19,41.14,672.17,6.0,3.0
3,2023-06-03 14:00:00+07:00,30,72.42,10.62,33.35,45.56,681.83,6.0,3.0
4,2023-06-03 15:00:00+07:00,30,64.14,8.93,34.08,49.41,718.33,6.0,3.0
...,...,...,...,...,...,...,...,...,...
1300,2023-10-15 14:00:00+07:00,30,72.03,10.23,32.91,36.37,704.17,10.0,15.0
1301,2023-10-15 15:00:00+07:00,30,65.11,8.97,32.58,36.16,725.83,10.0,15.0
1302,2023-10-15 16:00:00+07:00,30,71.59,9.64,30.96,38.45,742.30,10.0,15.0
1303,2023-10-15 17:00:00+07:00,30,89.13,11.78,28.76,42.89,756.80,10.0,15.0


In [ ]:
path = '/content/gdrive/My Drive/SNI/GKU/CO2/Hari/'


x=1
while x<=12 :
  y=1
  while y<=31:
    bul = str(x)
    har = str(y)
    dfhari = df1hco2[(df1hco2['Bulan']==x) & (df1hco2['Hari']==y) ]

    data = len(dfhari.axes[0])
    jumlah = str(data)
    if (dfhari.empty==True):
      pass
    else :
      std = dfhari['CO2'].std(numeric_only=True).round(2)
      mean = dfhari['CO2'].mean()
      cv = ((std / mean)*100).round(2)
      sd = str(std)
      koef = str(cv)
      dfhari.to_csv(path+"Bulan "+bul+" Hari "+har+" | "+"SD: "+sd+" ppm "+ "CV: " +koef+"% | "+"CO2 Validation.csv", index=None)
    y+=1
  x+=1

In [ ]:
# 1H in Months
path = '/content/gdrive/My Drive/SNI/GKU/CO2/Bulan/'

x=1
while x<=12 :

  bul = str(x)
  dfbulan = df1hco2[(df1hco2['Bulan']==x)]
  data = len(dfbulan.axes[0])
  jumlah = str(data)
  if (dfbulan.empty==True):
    pass
  else :
    std = dfbulan['CO2'].std(numeric_only=True).round(2)
    mean = dfbulan['CO2'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    dfbulan.to_csv(path+"Bulan "+bul+ " | "+"SD: "+sd+" ppm "+ "CV: " +koef+"% | "+"CO2 Validation 1H.csv", index=None)
  x+=1

In [ ]:
df8hco2 = df1hco2
df8hco2 = df8hco2.groupby(pd.Grouper(key='created_at',freq='8H')).mean().round(2).reset_index()
df8hco2 = df8hco2.loc[:,['created_at','σ (ppm)','CV (%)','Temperatur','Kelembapan Relatif','CO2','Bulan','Hari']]



In [ ]:
df8hco2

,created_at,σ (ppm),CV (%),Temperatur,Kelembapan Relatif,CO2,Bulan,Hari
0,2023-06-03 08:00:00+07:00,64.48,9.61,34.55,44.78,672.89,6.0,3.0
1,2023-06-03 16:00:00+07:00,69.33,8.30,25.22,83.13,839.61,6.0,3.0
2,2023-06-04 00:00:00+07:00,72.59,8.18,23.19,87.86,894.59,6.0,4.0
3,2023-06-04 08:00:00+07:00,72.82,10.86,33.18,52.48,674.44,6.0,4.0
4,2023-06-04 16:00:00+07:00,79.66,9.88,25.11,83.54,808.24,6.0,4.0
...,...,...,...,...,...,...,...,...
399,2023-10-14 08:00:00+07:00,79.23,11.58,32.29,44.85,688.09,10.0,14.0
400,2023-10-14 16:00:00+07:00,74.26,9.07,26.32,63.00,815.87,10.0,14.0
401,2023-10-15 00:00:00+07:00,72.53,8.52,23.22,79.62,855.17,10.0,15.0
402,2023-10-15 08:00:00+07:00,74.49,10.79,32.62,42.28,692.73,10.0,15.0


In [ ]:
# 8H in Months
path = '/content/gdrive/My Drive/SNI/GKU/CO2/Bulan/'

x=1
while x<=12 :

  bul = str(x)
  dfbulan = df8hco2[(df8hco2['Bulan']==x)]
  data = len(dfbulan.axes[0])
  jumlah = str(data)
  if (dfbulan.empty==True):
    pass
  else :
    std = dfbulan['CO2'].std(numeric_only=True).round(2)
    mean = dfbulan['CO2'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    dfbulan.to_csv(path+"Bulan "+bul+ " | "+"SD: "+sd+" ppm "+ "CV: " +koef+"% | "+"CO2 Validation 8H.csv", index=None)
  x+=1

Nah, disini tuh seharusnya kita validasi langsung aja per jam dulu cari outlier, terus kalau data per jamnya kurang dari 22.5 (75% perjam) di drop di jam itu, kalau udah, baru kita sorting.

# PM2.5

In [ ]:
#Validasi PM2.5


dfpm = df1.groupby(pd.Grouper(key='created_at',freq='2min')).mean().reset_index()

#Hapus nilai PM2.5 yang di luar batas range
dfpm = dfpm[(dfpm['PM2.5']>0) &  (dfpm['PM2.5'] <250)]

#Correction factor for RH >= 80
dfpm.loc[dfpm['Kelembapan Relatif']>=80, ['PM2.5']] *= 0.67

dfpm = dfpm.loc[:,['created_at','Temperatur','Kelembapan Relatif','PM2.5','Bulan','Hari','Jam']]

dfpm = dfpm.dropna()

# astype int ngubah angka ke int
dfpm['PM2.5']=dfpm['PM2.5'].round().astype(int)
dfpm['Bulan']=dfpm['Bulan'].round().astype(int)
dfpm['Hari']=dfpm['Hari'].round().astype(int)
dfpm['Jam']=dfpm['Jam'].round().astype(int)

dfpm

,created_at,Temperatur,Kelembapan Relatif,PM2.5,Bulan,Hari,Jam
14,2023-06-03 10:12:00+07:00,26.3,74.60,21,6,3,10
15,2023-06-03 10:14:00+07:00,27.1,72.80,123,6,3,10
21,2023-06-03 10:26:00+07:00,32.8,54.55,86,6,3,10
22,2023-06-03 10:28:00+07:00,33.2,53.00,89,6,3,10
33,2023-06-03 10:50:00+07:00,32.3,53.30,18,6,3,10
...,...,...,...,...,...,...,...
96773,2023-10-15 19:30:00+07:00,25.9,60.20,38,10,15,19
96774,2023-10-15 19:32:00+07:00,25.8,60.50,38,10,15,19
96775,2023-10-15 19:34:00+07:00,25.8,60.60,33,10,15,19
96776,2023-10-15 19:36:00+07:00,25.8,60.60,33,10,15,19


In [ ]:
dfcount = dfpm
dfstd = dfpm


dfcount = dfcount.groupby(pd.Grouper(key='created_at',freq='1H')).count()[lambda x: x > 22].dropna().round().astype(int)
# ambil 1 parameter hanya untuk menghitung ketersediaan data
dfcount = dfcount['PM2.5'].reset_index()
dfcount = dfcount.rename(columns={'PM2.5':'(n)Data'})

# hitung std PM2.5
dfstd = dfstd.groupby(pd.Grouper(key='created_at',freq='1H')).std()
dfstd = dfstd['PM2.5'].reset_index().round(2)
dfstd = dfstd.rename(columns={'PM2.5':'σ (µgr/m3)'})

dfdata = pd.merge(dfcount, dfstd, on='created_at', how='inner')



# ambil 1 parameter

In [ ]:
# Gabungkan data per jam hanya dengan data yang tersedia 75%
df1hpm = dfpm
df1hpm = df1hpm.groupby(pd.Grouper(key='created_at',freq='1H')).mean().round(2)

df1hpm = pd.merge(dfdata, df1hpm, on='created_at', how='inner')

df1hpm['CV (%)'] = ((df1hpm['σ (µgr/m3)'] / df1hpm['PM2.5']) * 100).round(2)

df1hpm = df1hpm.loc[:,['created_at','(n)Data','σ (µgr/m3)','CV (%)','Temperatur','Kelembapan Relatif','PM2.5','Bulan','Hari']]


df1hpm

,created_at,(n)Data,σ (µgr/m3),CV (%),Temperatur,Kelembapan Relatif,PM2.5,Bulan,Hari
0,2023-06-03 11:00:00+07:00,30,6.31,13.46,34.78,45.43,46.87,6.0,3.0
1,2023-06-03 12:00:00+07:00,30,1.63,3.37,35.34,42.35,48.33,6.0,3.0
2,2023-06-03 13:00:00+07:00,30,2.46,5.52,35.19,41.14,44.53,6.0,3.0
3,2023-06-03 14:00:00+07:00,30,0.00,0.00,33.35,45.56,44.00,6.0,3.0
4,2023-06-03 15:00:00+07:00,30,0.00,0.00,34.08,49.41,44.00,6.0,3.0
...,...,...,...,...,...,...,...,...,...
1250,2023-10-15 12:00:00+07:00,26,7.59,39.22,34.34,35.85,19.35,10.0,15.0
1251,2023-10-15 13:00:00+07:00,30,3.36,20.49,34.81,34.19,16.40,10.0,15.0
1252,2023-10-15 14:00:00+07:00,30,3.93,27.68,32.91,36.37,14.20,10.0,15.0
1253,2023-10-15 15:00:00+07:00,30,3.32,25.34,32.58,36.16,13.10,10.0,15.0


In [ ]:
df1hpm

,created_at,(n)Data,σ (µgr/m3),CV (%),Temperatur,Kelembapan Relatif,PM2.5,Bulan,Hari
0,2023-06-03 11:00:00+07:00,30,6.31,13.46,34.78,45.43,46.87,6.0,3.0
1,2023-06-03 12:00:00+07:00,30,1.63,3.37,35.34,42.35,48.33,6.0,3.0
2,2023-06-03 13:00:00+07:00,30,2.46,5.52,35.19,41.14,44.53,6.0,3.0
3,2023-06-03 14:00:00+07:00,30,0.00,0.00,33.35,45.56,44.00,6.0,3.0
4,2023-06-03 15:00:00+07:00,30,0.00,0.00,34.08,49.41,44.00,6.0,3.0
...,...,...,...,...,...,...,...,...,...
1250,2023-10-15 12:00:00+07:00,26,7.59,39.22,34.34,35.85,19.35,10.0,15.0
1251,2023-10-15 13:00:00+07:00,30,3.36,20.49,34.81,34.19,16.40,10.0,15.0
1252,2023-10-15 14:00:00+07:00,30,3.93,27.68,32.91,36.37,14.20,10.0,15.0
1253,2023-10-15 15:00:00+07:00,30,3.32,25.34,32.58,36.16,13.10,10.0,15.0


In [ ]:
path = '/content/gdrive/My Drive/SNI/GKU/PM2.5/Hari/'


x=1
while x<=12 :
  y=1
  while y<=31:
    bul = str(x)
    har = str(y)
    dfhari = df1hpm[(df1hpm['Bulan']==x) & (df1hpm['Hari']==y) ]

    data = len(dfhari.axes[0])
    jumlah = str(data)
    if (dfhari.empty==True):
      pass
    else :
      std = dfhari['PM2.5'].std(numeric_only=True).round(2)
      mean = dfhari['PM2.5'].mean()
      cv = ((std / mean)*100).round(2)
      sd = str(std)
      koef = str(cv)
      dfhari.to_csv(path+"Bulan "+bul+" Hari "+har+" | "+"SD: "+sd+" µgr.m3^-1 "+ "CV: " +koef+"% | "+"PM2.5 Validation.csv", index=None)
    y+=1
  x+=1

In [ ]:
# 1H in Months
path = '/content/gdrive/My Drive/SNI/GKU/PM2.5/Bulan/'

x=1
while x<=12 :

  bul = str(x)
  dfbulan = df1hpm[(df1hpm['Bulan']==x)]
  data = len(dfbulan.axes[0])
  jumlah = str(data)
  if (dfbulan.empty==True):
    pass
  else :
    std = dfbulan['PM2.5'].std(numeric_only=True).round(2)
    mean = dfbulan['PM2.5'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    dfbulan.to_csv(path+"Bulan "+bul+ " | "+"SD: "+sd+" µgr.m3^-1 "+ "CV: " +koef+"% | "+"PM2.5 Validation 1H.csv", index=None)
  x+=1

In [ ]:
df8hpm = df1hpm
df8hpm = df8hpm.groupby(pd.Grouper(key='created_at',freq='8H')).mean().round(2).reset_index()
df8hpm = df8hpm.loc[:,['created_at','σ (µgr/m3)','CV (%)','Temperatur','Kelembapan Relatif','PM2.5','Bulan','Hari']]



In [ ]:
# 8H in Months
path = '/content/gdrive/My Drive/SNI/GKU/PM2.5/Bulan/'

x=1
while x<=12 :

  bul = str(x)
  dfbulan = df8hpm[(df8hpm['Bulan']==x)]
  data = len(dfbulan.axes[0])
  jumlah = str(data)
  if (dfbulan.empty==True):
    pass
  else :
    std = dfbulan['PM2.5'].std(numeric_only=True).round(2)
    mean = dfbulan['PM2.5'].mean()
    cv = ((std / mean)*100).round(2)
    sd = str(std)
    koef = str(cv)
    dfbulan.to_csv(path+"Bulan "+bul+ " | "+"SD: "+sd+" µgr.m3^-1 "+ "CV: " +koef+"% | "+"PM2.5 Validation 8H.csv", index=None)
  x+=1